In [1]:
from os import chdir
chdir('../')
%matplotlib inline
from lib import *

pd.options.display.precision = 4
import matplotlib as mpl
mpl.rcParams['font.family'] = 'Lato'
mpl.rcParams['font.weight'] = 700
sns.set(font='Lato', font_scale=1)
sns.set()
np.random.seed(42)
suppress_warnings()

In [2]:
feature = pd.read_pickle('/home/jovyan/work/DSI/projects/project_3/data/feature')
target= pd.read_pickle('/home/jovyan/work/DSI/projects/project_3/data/target')

In [3]:
display(feature.shape)
display(target.shape)

(10768, 1000)

(10768, 1)

In [4]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, ShuffleSplit
from sklearn.feature_selection import SelectKBest, SelectPercentile, f_classif
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.decomposition import PCA
from sklearn.linear_model import Lasso, LogisticRegression,LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

# Step 2 Identify Features

In [5]:
from sklearn.feature_selection import SelectKBest, \
                                      SelectFromModel, \
                                      RFE, SelectPercentile, \
                                      SelectFdr, SelectFpr
from sklearn.feature_selection import f_regression, f_classif


### Pipeline: Select K best - PCA - SelectFromModel(RFC) - Logistic Regression

In [6]:
pipe_for_gs = Pipeline([
    ('skb', SelectKBest(score_func=f_classif)),
    ('pca',PCA(random_state=42)),
    ('sfmlr', SelectFromModel(LogisticRegression(C=0.1, penalty='l1',random_state=42))),
    ('sfmrf', SelectFromModel(RandomForestClassifier(random_state=42,n_jobs=-1))),
    ('lr', LogisticRegression(C=1E10,n_jobs=-1,random_state=42))
])

In [7]:
params = {
    'skb__k':[200,300,500],
    'pca__n_components':[50,100,150],
    'sfmrf__estimator':[  RandomForestClassifier(n_estimators=10, criterion='entropy',random_state=42,n_jobs=-1),
                          RandomForestClassifier(n_estimators=20, criterion='entropy',random_state=42,n_jobs=-1),  
                          RandomForestClassifier(n_estimators=30, criterion='entropy',random_state=42,n_jobs=-1),
                          RandomForestClassifier(n_estimators=40, criterion='entropy',random_state=42,n_jobs=-1),  
                          RandomForestClassifier(n_estimators=50, criterion='entropy',random_state=42,n_jobs=-1),
                        ],                    
}

In [8]:
gspipe = GridSearchCV(pipe_for_gs,
                      param_grid=params,
                      n_jobs=-1,
                      cv=ShuffleSplit(random_state=42))

In [9]:
gspipe.fit(feature[:220], np.ravel(target[:220].values))

GridSearchCV(cv=ShuffleSplit(n_splits=10, random_state=42, test_size=0.1, train_size=None),
       error_score='raise',
       estimator=Pipeline(steps=[('skb', SelectKBest(k=10, score_func=<function f_classif at 0x7f27da5db6a8>)), ('pca', PCA(copy=True, iterated_power='auto', n_components=None, random_state=42,
  svd_solver='auto', tol=0.0, whiten=False)), ('sfmlr', SelectFromModel(estimator=LogisticRegression(C=0.1, class_weight=None, dual=...alty='l2', random_state=42,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False))]),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'pca__n_components': [50, 100, 150], 'sfmrf__estimator': [RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples..._score=False, random_state=42,
            verbose=0, warm_start=False)], 'skb__k': [200, 300, 

In [10]:
gspipe.best_score_

0.52272727272727271

In [11]:
gspipe.best_estimator_.named_steps

{'lr': LogisticRegression(C=10000000000.0, class_weight=None, dual=False,
           fit_intercept=True, intercept_scaling=1, max_iter=100,
           multi_class='ovr', n_jobs=-1, penalty='l2', random_state=42,
           solver='liblinear', tol=0.0001, verbose=0, warm_start=False),
 'pca': PCA(copy=True, iterated_power='auto', n_components=50, random_state=42,
   svd_solver='auto', tol=0.0, whiten=False),
 'sfmlr': SelectFromModel(estimator=LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
           intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
           penalty='l1', random_state=42, solver='liblinear', tol=0.0001,
           verbose=0, warm_start=False),
         prefit=False, threshold=None),
 'sfmrf': SelectFromModel(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
             max_depth=None, max_features='auto', max_leaf_nodes=None,
             min_impurity_split=1e-07, min_samples_leaf=1,
 

In [12]:
gspipe_results = pd.DataFrame(gspipe.cv_results_)
gspipe_results.sort_values('rank_test_score', ascending=True).T

,14,8,11,2,35,20,1,5,38,41,...,19,10,24,39,12,36,21,27,42,9
mean_fit_time,0.1352,0.1017,0.1124,0.05989,0.0803,0.0799,0.05015,0.07651,0.108,0.1244,...,0.06682,0.1009,0.0934,0.09596,0.1067,0.07659,0.07672,0.1177,0.1102,0.08972
mean_score_time,0.003724,0.002641,0.003128,0.001682,0.002258,0.002324,0.001379,0.001984,0.003188,0.003369,...,0.002034,0.002967,0.00387,0.003119,0.003483,0.002829,0.002424,0.003796,0.003553,0.002953
mean_test_score,0.5227,0.5045,0.4955,0.4909,0.4909,0.4864,0.4864,0.4864,0.4818,0.4818,...,0.4182,0.4136,0.4136,0.4136,0.4091,0.4045,0.4045,0.4,0.4,0.3909
mean_train_score,0.7662,0.7732,0.7712,0.7525,0.8288,0.8152,0.8247,0.7616,0.8561,0.8485,...,0.847,0.8146,0.8449,0.8449,0.8323,0.8485,0.8485,0.8424,0.8424,0.8359
param_pca__n_components,50,50,50,50,150,100,50,50,150,150,...,100,50,100,150,50,150,100,100,150,50
param_sfmrf__estimator,(),(),(),(),(),(),(),(),(),(),...,(),(),(),(),(),(),(),(),(),()
param_skb__k,500,500,500,500,500,500,300,500,500,500,...,300,300,200,200,200,200,200,200,200,200
params,"{'pca__n_components': 50, 'sfmrf__estimator': ...","{'pca__n_components': 50, 'sfmrf__estimator': ...","{'pca__n_components': 50, 'sfmrf__estimator': ...","{'pca__n_components': 50, 'sfmrf__estimator': ...","{'pca__n_components': 150, 'sfmrf__estimator':...","{'pca__n_components': 100, 'sfmrf__estimator':...","{'pca__n_components': 50, 'sfmrf__estimator': ...","{'pca__n_components': 50, 'sfmrf__estimator': ...","{'pca__n_components': 150, 'sfmrf__estimator':...","{'pca__n_components': 150, 'sfmrf__estimator':...",...,"{'pca__n_components': 100, 'sfmrf__estimator':...","{'pca__n_components': 50, 'sfmrf__estimator': ...","{'pca__n_components': 100, 'sfmrf__estimator':...","{'pca__n_components': 150, 'sfmrf__estimator':...","{'pca__n_components': 50, 'sfmrf__estimator': ...","{'pca__n_components': 150, 'sfmrf__estimator':...","{'pca__n_components': 100, 'sfmrf__estimator':...","{'pca__n_components': 100, 'sfmrf__estimator':...","{'pca__n_components': 150, 'sfmrf__estimator':...","{'pca__n_components': 50, 'sfmrf__estimator': ..."
rank_test_score,1,2,3,4,4,6,6,6,9,9,...,34,37,37,37,40,41,41,43,43,45
split0_test_score,0.5,0.5909,0.5455,0.5909,0.5455,0.5,0.4091,0.4091,0.5455,0.5909,...,0.4545,0.4545,0.4545,0.4545,0.4091,0.4545,0.4545,0.4545,0.4545,0.4091


In [13]:
gspipe_results['mean_fit_time'].sum()

4.0366488695144653

In [14]:
gspipe_results.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45 entries, 0 to 44
Data columns (total 33 columns):
mean_fit_time              45 non-null float64
mean_score_time            45 non-null float64
mean_test_score            45 non-null float64
mean_train_score           45 non-null float64
param_pca__n_components    45 non-null object
param_sfmrf__estimator     45 non-null object
param_skb__k               45 non-null object
params                     45 non-null object
rank_test_score            45 non-null int32
split0_test_score          45 non-null float64
split0_train_score         45 non-null float64
split1_test_score          45 non-null float64
split1_train_score         45 non-null float64
split2_test_score          45 non-null float64
split2_train_score         45 non-null float64
split3_test_score          45 non-null float64
split3_train_score         45 non-null float64
split4_test_score          45 non-null float64
split4_train_score         45 non-null float64
split5_tes

In [15]:
print("Fit time .1% data: ", gspipe_results['mean_fit_time'].sum())
print("Best Score .1% data: ",gspipe.best_score_)
### Pipeline3 for .1% (220 rows)
picked_pipeline_220row = \
        pd.DataFrame.to_pickle(gspipe_results,\
        '/home/jovyan/work/DSI/projects/project_3/data/Pipeline3_lr_pt1_percent')

Fit time .1% data:  4.03664886951
Best Score .1% data:  0.522727272727


In [16]:
### Pipeline3 for .5% (1100 rows)

gspipe.fit(feature[:1100], np.ravel(target[:1100].values))
gspipe_results = pd.DataFrame(gspipe.cv_results_)
print("Fit time .5% data: ", gspipe_results['mean_fit_time'].sum())
print("Best Score .5% data: ",gspipe.best_score_)
gspipe_results_1100 = pd.DataFrame(gspipe.cv_results_)
picked_Pipeline3_1100row = \
        pd.DataFrame.to_pickle(gspipe_results_1100,\
        '/home/jovyan/work/DSI/projects/project_3/data/Pipeline3_lr_pt5_percent')

Fit time .5% data:  21.4086084127
Best Score .5% data:  0.522727272727


In [17]:
### Pipeline3 for 1% (2200 rows)

gspipe.fit(feature[:2200], np.ravel(target[:2200].values))
gspipe_results = pd.DataFrame(gspipe.cv_results_)
print("Fit time 1% data: ", gspipe_results['mean_fit_time'].sum())
print("Best Score 1% data: ",gspipe.best_score_)
gspipe_results_2200 = pd.DataFrame(gspipe.cv_results_)
picked_Pipeline2_2200row = \
        pd.DataFrame.to_pickle(gspipe_results_2200,\
        '/home/jovyan/work/DSI/projects/project_3/data/Pipeline3_lr_1_percent')

Fit time 1% data:  48.9192883253
Best Score 1% data:  0.53


In [18]:
### Bench Marking for 5% (11000 rows)

gspipe.fit(feature, np.ravel(target.values))
gspipe_results = pd.DataFrame(gspipe.cv_results_)
print("Fit time 5% data: ", gspipe_results['mean_fit_time'].sum())
print("Best Score 5% data:",gspipe.best_score_)
gspipe_results_11000 = pd.DataFrame(gspipe.cv_results_)
picked_GS_BenchMarkig_11000row = \
        pd.DataFrame.to_pickle(gspipe_results_11000,\
        '/home/jovyan/work/DSI/projects/project_3/data/Pipeline3_lr_5_percent')

Fit time 5% data:  351.047479606
Best Score 5% data: 0.527855153203
